<a href="https://colab.research.google.com/github/marcinmichalsmolen/adam/blob/main/TFDS_FLOWERS_Z_VGG16_ZLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adagrad, RMSprop, SGD, Adadelta, Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load TFDS dataset
dataset_name = 'tf_flowers'
(train_dataset, validation_dataset, test_dataset), dataset_info = tfds.load(
    name=dataset_name,
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    shuffle_files=True,
    with_info=True,
    as_supervised=True
)

# Preprocess data
def preprocess(image, label):
    image = tf.image.resize(image, (150, 150)) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess)
validation_dataset = validation_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# Define model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_dataset.shape)
base_model.trainable = False ## Wagi nie będą trenowane


from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model_tfds = model1 = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])


# Define training settings
batch_size = 64
epochs = 30
optimizer_names = ['Adagrad', 'RMSprop', 'SGD', 'Adadelta', 'Adam']
optimizers = [Adagrad(), RMSprop(), SGD(), Adadelta(), Adam()]

# Train models with different optimizers
history = {}
for optimizer_name, optimizer in zip(optimizer_names, optimizers):
    print(f'Training model with {optimizer_name} optimizer...')
    model_tfds.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history[optimizer_name] = model_tfds.fit(train_dataset.batch(batch_size), epochs=epochs,
                                             validation_data=validation_dataset.batch(batch_size))

# Plot training loss for all models on one graph
plt.figure(figsize=(10, 6))
for optimizer_name in optimizer_names:
    plt.plot(history[optimizer_name].history['loss'], label=optimizer_name)
plt.title('Training Loss for Different Optimizers')
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
for optimizer_name in optimizer_names:
    plt.plot(history[optimizer_name].history['val_loss'], label=optimizer_name)
plt.title('Training Loss for Different Optimizers')
plt.xlabel('Epoch')
plt.ylabel('Test Loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
for optimizer_name in optimizer_names:
    plt.plot(history[optimizer_name].history['accuracy'], label=optimizer_name)
plt.title('Accuracy for Different Optimizers')
plt.xlabel('Epoch')
plt.ylabel('Training Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
for optimizer_name in optimizer_names:
    plt.plot(history[optimizer_name].history['val_accuracy'], label=optimizer_name)
plt.title('Val_Accuracy for Different Optimizers')
plt.xlabel('Epoch')
plt.ylabel('Val_Accuracy')
plt.legend()
plt.show()


AttributeError: ignored